In [12]:
from collections import defaultdict, Counter
import numpy as np
import pandas as pd

In [13]:
class LanguageNgramModel:
    """ Remember and predict which letters usually follows which. """
    def __init__(self, order=1, smoothing=1.0, recursive=0.001):
        self.order = order
        self.smoothing = smoothing
        self.recursive = recursive
    
    def fit(self, corpus):
        """ Estimate all counts on a text """
        self.counter_ = defaultdict(lambda: Counter())
        self.unigrams_ = Counter()
        self.vocabulary_ = set()
        for i, token in enumerate(corpus[self.order:]):
            context = corpus[i:(i+self.order)]
            self.counter_[context][token] += 1
            self.unigrams_[token] +=1
            self.vocabulary_.add(token)
        self.vocabulary_ = sorted(list(self.vocabulary_))
        if self.recursive > 0 and self.order > 0:
            self.child_ = LanguageNgramModel(self.order-1, self.smoothing, self.recursive)
            self.child_.fit(corpus)
            
    def get_counts(self, context):
        """ Get smoothed count of each letter appearing after context """
        if self.order:
            local = context[-self.order:]
        else:
            local = ''
        freq_dict = self.counter_[local]
        freq = pd.Series(index=self.vocabulary_)
        for i, token in enumerate(self.vocabulary_):
            freq[token] = freq_dict[token] + self.smoothing
        if self.recursive > 0 and self.order > 0:
            child_freq = self.child_.get_counts(context) * self.recursive
            freq += child_freq
        return freq
    
    def predict_proba(self, context):
        """ Get smoothed probability of each letter appearing after context """
        counts = self.get_counts(context)
        return counts / counts.sum()
    
    def single_log_proba(self, context, continuation):
        """ Estimate log-probability that context is followed by continuation """
        result = 0.0
        for token in continuation:
            result += np.log(self.predict_proba(context)[token])
            context += token
        return result
    
    def single_proba(self, context, continuation):
        """ Estimate probability that context is followed by continuation """
        return np.exp(self.single_log_proba(context, continuation))

In [14]:
class MissingLetterModel:
    """ Remember and predict which letters are usually missing. """
    def __init__(self, order=0, smoothing_missed=0.3, smoothing_total=1.0):
        self.order = order
        self.smoothing_missed = smoothing_missed
        self.smoothing_total = smoothing_total
    def fit(self, sentence_pairs):
        self.missed_counter_ = defaultdict(lambda: Counter())
        self.total_counter_ = defaultdict(lambda: Counter())
        for (original, observed) in sentence_pairs:
            for i, (original_letter, observed_letter) in enumerate(zip(original[self.order:], observed[self.order:])):
                context = original[i:(i+self.order)]
                if observed_letter == '-':
                    self.missed_counter_[context][original_letter] += 1
                self.total_counter_[context][original_letter] += 1 
    def predict_proba(self, context, last_letter):
        """ Estimate probability that last_letter after context is missed """
        if self.order:
            local = context[-self.order:]
        else:
            local = ''
        missed_freq = self.missed_counter_[local][last_letter] + self.smoothing_missed
        total_freq = self.total_counter_[local][last_letter] + self.smoothing_total
        return missed_freq / total_freq
    
    def single_log_proba(self, context, continuation, actual=None):
        """ Estimate log-probability of continuaton being distorted to actual after context. 
        If actual is None, assume no distortion
        """
        if not actual:
            actual = continuation
        result = 0.0
        for orig_token, act_token in zip(continuation, actual):
            pp = self.predict_proba(context, orig_token)
            if act_token == '-':
                pp = 1 - pp
            result += np.log(pp)
            context += orig_token
        return result
    
    def single_proba(self, context, continuation, actual=None):
        """ Estimate probability of continuaton being distorted to actual after context. 
        If actual is None, assume no distortion
        """
        return np.exp(self.single_log_proba(context, continuation, actual))

In [15]:
from heapq import heappush, heappop

In [16]:
def generate_options(prefix_proba, prefix, suffix, lang_model, missed_model, optimism=0.5, cache=None):
    options = []
    for letter in lang_model.vocabulary_ + ['']:
        if letter:  # assume a missing letter
            next_letter = letter
            new_suffix = suffix
            new_prefix = prefix + next_letter
            proba_missing_state = - np.log(missed_model.predict_proba(prefix, letter))
        else:  # assume no missing letter
            next_letter = suffix[0]
            new_suffix = suffix[1:]
            new_prefix = prefix + next_letter
            proba_missing_state = - np.log((1 - missed_model.predict_proba(prefix, next_letter)))
        proba_next_letter = - np.log(lang_model.single_proba(prefix, next_letter))
        if cache:
            proba_suffix = cache[len(new_suffix)] * optimism
        else:
            proba_suffix = - np.log(lang_model.single_proba(new_prefix, new_suffix)) * optimism
        proba = prefix_proba + proba_next_letter + proba_missing_state + proba_suffix
        options.append((proba, new_prefix, new_suffix, letter, proba_suffix))
    return options

In [17]:
def noisy_channel(word, lang_model, missed_model, freedom=1.0, max_attempts=1000, optimism=0.1, verbose=True):
    query = word + ' '
    prefix = ' '
    prefix_proba = 0.0
    suffix = query
    full_origin_logprob = -lang_model.single_log_proba(prefix, query)
    no_missing_logprob = -missed_model.single_log_proba(prefix, query)
    best_logprob = full_origin_logprob + no_missing_logprob
    # add empty beginning to the heap
    heap = [(best_logprob * optimism, prefix, suffix, '', best_logprob * optimism)]
    # add the default option (no missing letters) to candidates
    candidates = [(best_logprob, prefix + query, '', None, 0.0)]
    if verbose:
        # todo: include distortion probability
        print('baseline score is', best_logprob)
    # prepare cache for suffixes (the slowest operation)
    cache = {}
    for i in range(len(query)+1):
        future_suffix = query[:i]
        cache[len(future_suffix)] = -lang_model.single_log_proba('', future_suffix) # rough approximation
        cache[len(future_suffix)] += -missed_model.single_log_proba('', future_suffix) # at least add missingness
    
    for i in range(max_attempts):
        if not heap:
            break
        next_best = heappop(heap)
        if verbose:
            print(next_best)
        if next_best[2] == '':  # it is a leaf
            # this is the best leaf as far, add it to candidates
            if next_best[0] <= best_logprob + freedom:
                candidates.append(next_best)
                # update the best likelihood
                if next_best[0] < best_logprob:
                    best_logprob = next_best[0]
        else: # it is not a leaf - generate more options
            prefix_proba = next_best[0] - next_best[4] # all proba estimate minus suffix
            prefix = next_best[1]
            suffix = next_best[2]
            new_options = generate_options(prefix_proba, prefix, suffix, lang_model, missed_model, optimism, cache)
            # add only the solution potentioally no worse than the best + freedom
            for new_option in new_options: 
                if new_option[0] < best_logprob + freedom:
                    heappush(heap, new_option)
    if verbose:
        print('heap size is', len(heap), 'after', i, 'iterations')
    result = {}
    for candidate in candidates:
        if candidate[0] <= best_logprob + freedom:
            result[candidate[1][1:-1]] = candidate[0]
    return result

In [18]:
with open('accounting terms.txt', encoding = 'utf-8') as f:
    text = f.read()
import re
text2 = re.sub(r'[^a-z ]+', '', text.lower().replace('\n', ' '))
print(text2[0:1000])

k plan employee benefit plan authorized by internal revenue code section k whereby an employer establishes an account for each participating employee and each participant elects to deposit a portion of his or her salary into the account the amount deposited is not subject to income tax this is the most common type of salary reduction plans  a misstatement is inconsequential if a reasonable person would conclude after considering the possibility of further undetected misstatements that the misstatement either individually or when aggregated with other misstatements would clearly be immaterial to the financial statements if a reasonable person could not reach such a conclusion regarding a particular misstatement that misstatement is more than inconsequential  abatement complete removal of an amount due usually referring to a tax abatement a penalty abatement or an interest abatement within a governing agency   absorption costing an approach to product costing that assigns a representativ

In [19]:
all_letters = ''.join(list(sorted(list(set(text2)))))
print(repr(all_letters))

' abcdefghijklmnopqrstuvwxyz'


In [45]:
all_letters

' abcdefghijklmnopqrstuvwxyz'

In [20]:
missing_set = [] + [(all_letters, '-' * len(all_letters))] * 3 + [(all_letters, all_letters)] * 10 + [('aeiouy', '------')] * 30

In [11]:
for i in range(5):
    tmp = LanguageNgramModel(i, 1.0, 0.001)
    tmp.fit(text2[0:-5000])
    print(i, tmp.single_log_proba(' ', text2[-5000:]))

0 -14241.407785007828
1 -11669.625521970873
2 -9131.981344709282
3 -7495.287810769386
4 -7860.915675167994


In [22]:
big_lang_m = LanguageNgramModel(4, 0.001, 0.01)
big_lang_m.fit(text2)
big_err_m = MissingLetterModel(0, 0.1)
big_err_m.fit(missing_set)

In [17]:
noisy_channel('equip', big_lang_m, big_err_m, max_attempts=10000, optimism=0.9, freedom=3.0, verbose=False)

{'equip': 21.06765260437274,
 'equipme': 21.488484758096885,
 'equipment': 19.505340909496294}

In [18]:
noisy_channel('avg', big_lang_m, big_err_m, max_attempts=10000, optimism=0.9, freedom=3.0, verbose=False)

{'average': 12.788721504012619}

In [19]:
noisy_channel('val', big_lang_m, big_err_m, max_attempts=10000, optimism=0.9, freedom=3.0, verbose=False)

{'value': 8.159271854013921}

In [20]:
noisy_channel('htl', big_lang_m, big_err_m, max_attempts=10000, optimism=0.9, freedom=3.0, verbose=False)

{'htl': 15.969590860546486,
 'htle': 16.778589734463562,
 'htline': 14.868173966337155,
 'htly': 15.620779305580742}

In [21]:
noisy_channel('lbr', big_lang_m, big_err_m, max_attempts=10000, optimism=0.9, freedom=3.0, verbose=False)

{'labor': 9.396400299087876}

In [22]:
noisy_channel('accum', big_lang_m, big_err_m, max_attempts=10000, optimism=0.9, freedom=3.0, verbose=False)

{'accum': 20.32462037011524,
 'accumular': 19.887297345300013,
 'accumulate': 18.33715931588073,
 'accumulated': 19.210050111412855,
 'accumulatio': 21.062298793171916}

In [ ]:
noisy_channel('discnt', big_lang_m, big_err_m, max_attempts=10000, optimism=0.9, freedom=3.0, verbose=False)

{'discount': 11.61182032238721, 'discounts': 14.302659691836924}

In [24]:
noisy_channel('compl', big_lang_m, big_err_m, max_attempts=10000, optimism=0.9, freedom=3.0, verbose=False)

{'comple': 11.488833646106656,
 'comples': 14.380924742921977,
 'complex': 14.351387722031802,
 'comply': 13.045891808168188}

In [ ]:
noisy_channel('misc', big_lang_m, big_err_m, max_attempts=10000, optimism=0.9, freedom=3.0, verbose=False)

In [51]:
generate_options(0,' ','avg',big_lang_m,big_err_m)

[(11.585656396051844, '  ', 'avg', ' ', 7.651135031499331),
 (9.400374515051253, ' a', 'avg', 'a', 6.6936431004705454),
 (11.40433271613345, ' b', 'avg', 'b', 6.5958596495427955),
 (11.34257925140478, ' c', 'avg', 'c', 6.758417758051705),
 (11.537429404562896, ' d', 'avg', 'd', 6.608516115946221),
 (10.291494484875397, ' e', 'avg', 'e', 6.6785643775338155),
 (11.872940015379823, ' f', 'avg', 'f', 7.0862189107289275),
 (11.445613683041893, ' g', 'avg', 'g', 6.176649389482181),
 (11.971284468517618, ' h', 'avg', 'h', 6.62981597519682),
 (9.82111837446565, ' i', 'avg', 'i', 6.654816767194983),
 (11.085179896390887, ' j', 'avg', 'j', 5.506740204516885),
 (10.942251160093887, ' k', 'avg', 'k', 5.379425282380009),
 (11.445661223609012, ' l', 'avg', 'l', 6.300229929023635),
 (11.455320430123496, ' m', 'avg', 'm', 6.473514885004395),
 (11.91756081177267, ' n', 'avg', 'n', 6.736044162484071),
 (9.606787373925366, ' o', 'avg', 'o', 6.6116911708420005),
 (11.345943253410141, ' p', 'avg', 'p', 6.7

In [39]:
noisy_channel('avg', big_lang_m, big_err_m, max_attempts=10000, optimism=0.9, freedom=3.0, verbose=True)

baseline score is 20.387755905172256
(18.34898031465503, ' ', 'avg ', '', 18.34898031465503)
(16.285630682073435, ' a', 'vg ', '', 12.468128774344809)
(16.471204972362393, ' av', 'g ', '', 7.786665074547269)
(17.315142567055396, ' ave', 'g ', 'e', 7.786665074547269)
(17.942191427989087, ' ava', 'g ', 'a', 7.786665074547269)
(18.066478524587417, ' avg', ' ', '', 3.1710464594368584)
(17.726444365088188, ' avg ', '', '', -0.0)
(18.23251967141732, ' avai', 'g ', 'i', 7.786665074547269)
(18.85012617720313, ' aver', 'g ', 'r', 7.786665074547269)
(18.854395868838978, ' a', 'avg ', 'a', 16.147664454258273)
(19.142760657341636, ' o', 'avg ', 'o', 16.147664454258273)
(19.190678827759832, ' an', 'vg ', 'n', 12.468128774344809)
(19.27244428280196, ' avera', 'g ', 'a', 7.786665074547269)
(15.168048418919401, ' averag', ' ', '', 3.1710464594368584)
(15.453771090656028, ' average', ' ', 'e', 3.1710464594368584)
(12.788721504012619, ' average ', '', '', -0.0)
(17.217128641193476, ' average ', ' ', ' '

{'average': 12.788721504012619}

In [46]:
lang_model=big_lang_m
missed_model=big_err_m
query = 'avg' + ' '
prefix = ' '
prefix_proba = 0.0
freedom=3.0
max_attempts=1000
optimism=0.9
verbose=True
suffix = query
full_origin_logprob = -lang_model.single_log_proba(prefix, query)
no_missing_logprob = -missed_model.single_log_proba(prefix, query)
best_logprob = full_origin_logprob + no_missing_logprob
print(best_logprob)
print(full_origin_logprob)
print(no_missing_logprob)

20.387755905172256
15.58013389926454
4.807622005907717


In [49]:
-lang_model.single_log_proba(prefix, query)

'avg '

In [31]:
heap = [(best_logprob * optimism, prefix, suffix, '', best_logprob * optimism)]
heap

[(18.34898031465503, ' ', 'avg ', '', 18.34898031465503)]

In [32]:
candidates = [(best_logprob, prefix + query, '', None, 0.0)]
candidates

[(20.387755905172256, ' avg ', '', None, 0.0)]

In [36]:
cache = {}
for i in range(len(query)+1):
    future_suffix = query[:i]
    print(future_suffix)
    cache[len(future_suffix)] = -lang_model.single_log_proba('', future_suffix) # rough approximation
    #print(cache)
    cache[len(future_suffix)] += -missed_model.single_log_proba('', future_suffix) # at least add missingness
    print(cache)


{0: -0.0}
a
{0: -0.0, 1: 3.5233849549298424}
av
{0: -0.0, 1: 3.5233849549298424, 2: 8.651850082830299}
avg
{0: -0.0, 1: 3.5233849549298424, 2: 8.651850082830299, 3: 13.853476415938676}
avg 
{0: -0.0, 1: 3.5233849549298424, 2: 8.651850082830299, 3: 13.853476415938676, 4: 17.941849393620302}


In [40]:
i=0
next_best = heappop(heap)
next_best

(18.34898031465503, ' ', 'avg ', '', 18.34898031465503)

In [44]:
print(next_best[2])
print(best_logprob)
if next_best[2] == '':
    print(next_best[2])
    if next_best[0] <= best_logprob + freedom:
        print(next_best)
        #candidates.append(next_best)
        # update the best likelihood
        if next_best[0] < best_logprob:
            print(next_best[0])
            #best_logprob = next_best[0]

avg 
20.387755905172256


In [ ]:
if next_best[2] == '':  # it is a leaf
    # this is the best leaf as far, add it to candidates
    if next_best[0] <= best_logprob + freedom:
        candidates.append(next_best)
        # update the best likelihood
        if next_best[0] < best_logprob:
            best_logprob = next_best[0]
else: # it is not a leaf - generate more options
    prefix_proba = next_best[0] - next_best[4] # all proba estimate minus suffix
    prefix = next_best[1]
    suffix = next_best[2]
    new_options = generate_options(prefix_proba, prefix, suffix, lang_model, missed_model, optimism, cache)
    # add only the solution potentioally no worse than the best + freedom
    for new_option in new_options: 
        if new_option[0] < best_logprob + freedom:
            heappush(heap, new_option)

In [ ]:
for i in range(max_attempts):
    if not heap:
        break
    next_best = heappop(heap)
    if verbose:
        print(next_best)
    if next_best[2] == '':  # it is a leaf
        # this is the best leaf as far, add it to candidates
        if next_best[0] <= best_logprob + freedom:
            candidates.append(next_best)
            # update the best likelihood
            if next_best[0] < best_logprob:
                best_logprob = next_best[0]
    else: # it is not a leaf - generate more options
        prefix_proba = next_best[0] - next_best[4] # all proba estimate minus suffix
        prefix = next_best[1]
        suffix = next_best[2]
        new_options = generate_options(prefix_proba, prefix, suffix, lang_model, missed_model, optimism, cache)
        # add only the solution potentioally no worse than the best + freedom
        for new_option in new_options: 
            if new_option[0] < best_logprob + freedom:
                heappush(heap, new_option)

In [ ]:
def noisy_channel(word, lang_model, missed_model, freedom=1.0, max_attempts=1000, optimism=0.1, verbose=True):
    query = word + ' '
    prefix = ' '
    prefix_proba = 0.0
    suffix = query
    full_origin_logprob = -lang_model.single_log_proba(prefix, query)
    no_missing_logprob = -missed_model.single_log_proba(prefix, query)
    best_logprob = full_origin_logprob + no_missing_logprob
    # add empty beginning to the heap
    heap = [(best_logprob * optimism, prefix, suffix, '', best_logprob * optimism)]
    # add the default option (no missing letters) to candidates
    candidates = [(best_logprob, prefix + query, '', None, 0.0)]
    if verbose:
        # todo: include distortion probability
        print('baseline score is', best_logprob)
    # prepare cache for suffixes (the slowest operation)
    cache = {}
    for i in range(len(query)+1):
        future_suffix = query[:i]
        cache[len(future_suffix)] = -lang_model.single_log_proba('', future_suffix) # rough approximation
        cache[len(future_suffix)] += -missed_model.single_log_proba('', future_suffix) # at least add missingness
    
    for i in range(max_attempts):
        if not heap:
            break
        next_best = heappop(heap)
        if verbose:
            print(next_best)
        if next_best[2] == '':  # it is a leaf
            # this is the best leaf as far, add it to candidates
            if next_best[0] <= best_logprob + freedom:
                candidates.append(next_best)
                # update the best likelihood
                if next_best[0] < best_logprob:
                    best_logprob = next_best[0]
        else: # it is not a leaf - generate more options
            prefix_proba = next_best[0] - next_best[4] # all proba estimate minus suffix
            prefix = next_best[1]
            suffix = next_best[2]
            new_options = generate_options(prefix_proba, prefix, suffix, lang_model, missed_model, optimism, cache)
            # add only the solution potentioally no worse than the best + freedom
            for new_option in new_options: 
                if new_option[0] < best_logprob + freedom:
                    heappush(heap, new_option)
    if verbose:
        print('heap size is', len(heap), 'after', i, 'iterations')
    result = {}
    for candidate in candidates:
        if candidate[0] <= best_logprob + freedom:
            result[candidate[1][1:-1]] = candidate[0]
    return result